In [158]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader('summary_results_zhanghua.csv')
data = loader.load()
 
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1024,
                                               chunk_overlap  = 100,
                                               length_function = len,
                                               is_separator_regex = False)

all_splits = text_splitter.split_documents(data)

In [159]:
from langchain.embeddings import HuggingFaceEmbeddings 
embeddings = HuggingFaceEmbeddings(model_name='shibing624/text2vec-base-chinese', 
                                   model_kwargs={'device': 'cuda'}) 

from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(data, embeddings)

In [160]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(model_path="/work/u9796576/Llama/quantized_q4_1.gguf",
               n_gpu_layers=-1,#If -1, all layers are offloaded.
               n_batch=256,
               n_ctx=0, #0 = from model
               f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
               verbose=False,
               temperature=0.1,
               top_k=50,
               top_p=0.95,
               stop=['\n', 'ENDINPUT','ENDINSTRUCTION'])

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /work/u9796576/Llama/quantized_q4_1.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_1     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_1     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_1     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_1     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_1     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_1     [  4096,  4096,  

In [171]:
from langchain.schema.retriever import BaseRetriever
from langchain_core.documents import Document
from typing import List
from langchain_core.callbacks.manager import CallbackManagerForRetrieverRun
import random

# Retriever with random sampling
class CustomRetriever(BaseRetriever):
    retriever: BaseRetriever
        
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        # Use your existing retriever to get the documents
        documents = self.retriever.get_relevant_documents(query, callbacks=run_manager.get_child())
        
        # print(documents)
        
        # Sort the documents by "source"
        # documents = sorted(documents, key=lambda doc: doc.metadata.get('source'))
        
        return [random.choice(documents)]
        # return (*random.sample(documents, 2), )

k_samples = 5
custom_retriever = CustomRetriever(retriever = vectorstore.as_retriever(search_kwargs={"k": k_samples}))

In [175]:
from langchain.chains import RetrievalQA
from langchain import PromptTemplate

emplate = """作為一個美食推薦家,你需要使用位於context中的內容去回答問題,且遵循rule中的規則。

rule
1.只要輸出簡潔的一句話就好
2.盡量不要輸出距離有多遠除非問題有提及遠近相關的詞彙
3.不要輸出ASSISTANT: 
endrule

context
{context}
endcontext

問題: {question}
回答: """

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)

# qa_chain = RetrievalQA.from_chain_type(llm,
#                                        chain_type='stuff',
#                                        retriever=vectorstore.as_retriever(search_kwargs={"k": 1}),
#                                        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT,
#                                                           "verbose": True,
#                                                           })

qa_chain = RetrievalQA.from_chain_type(llm,
                                       chain_type='stuff',
                                       retriever=custom_retriever,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT,
                                                          "verbose": True,
                                                          })

ask_model = lambda x, return_query=False: qa_chain({"query": x})['result'] if not return_query else \
                                          f"問題: {x}, 回答: {qa_chain({'query': x})['result']}"


In [173]:
print(ask_model('請推薦一間牛肉麵', return_query=True))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
作為一個美食推薦家,你需要使用位於context中的內容去回答問題,且遵循rule中的規則。

rule
1.只要輸出簡潔的一句話就好
2.盡量不要輸出距離有多遠除非問題有提及遠近相關的詞彙
3.不要輸出ASSISTANT: 
endrule

context
name: 老店牛肉麵
rating: 4.1
keyword: 麵食, 牛肉
distance: 1.2660131860872943
summary: 牛肉麵、滷味、小菜、辣椒醬、中天電視
endcontext

問題: 請推薦一間牛肉麵
回答: 

> Finished chain.

> Finished chain.
問題: 請推薦一間牛肉麵, 回答: 老店牛肉麵是一家位於市中心的小店,提供美味的牛肉麵和其他菜品。


In [ ]:
# print(ask_model('請推薦一間牛肉麵'))
# print(ask_model('請推薦一間賣串燒的店家'))
# print(ask_model('我想吃炸雞'))
# print(ask_model('這附近有沒有賣炒飯的'))
# print(ask_model('有沒有吃到飽'))
# print(ask_model('我想吃拉麵'))
# print(ask_model('安排個附近的素食店吧'))
# print(ask_model('想吃鬆餅要去哪裡買'))
# print(ask_model('推薦個附近的素食店吧'))
# print(ask_model('我想吃滷味'))
# print(ask_model('推薦良叔叔滷味的哪些餐點'))
# print(ask_model('幫我推薦一間便宜的滷味'))
# print(ask_model('麥當勞跟肯德基選哪個'))
# print(ask_model('有沒有外送串燒'))
# print(ask_model('好久沒吃燒臘了'))
# print(ask_model('胖老爹跟麥當勞哪個好'))
# print(ask_model('今天我想來點吃到飽'))
# print(ask_model('吃到飽安排上，哪間店推薦下'))
# print(ask_model('天氣冷該吃個火鍋了吧'))

In [ ]:
# print(qa_chain({"query": "請推薦一間牛肉麵"})['result'])
# print(qa_chain({"query": "請推薦一間賣串燒的店家"})['result'])
# print(qa_chain({"query": "我想吃炸雞"})['result'])
# print(qa_chain({"query": "這附近有沒有賣炒飯的"})['result'])
# print(qa_chain({"query": "有沒有吃到飽"})['result'])
# print(qa_chain({"query": "我想吃拉麵"})['result'])
# print(qa_chain({"query": "安排個附近的素食店吧"})['result'])
# print(qa_chain({"query": "想吃鬆餅要去哪裡買"})['result'])
# print(qa_chain({"query": "推薦個附近的素食店吧"})['result'])
# print(qa_chain({"query": "我想吃滷味"})['result'])
# print(qa_chain({"query": "推薦良叔叔滷味的哪些餐點"})['result'])
# print(qa_chain({"query": "幫我推薦一間便宜的滷味"})['result'])
# print(qa_chain({"query": "麥當勞跟肯德基選哪個"})['result'])
# print(qa_chain({"query": "有沒有外送串燒"})['result'])
# print(qa_chain({"query": "好久沒吃燒臘了"})['result'])
# print(qa_chain({"query": "胖老爹跟麥當勞哪個好"})['result'])
# print(qa_chain({"query": "今天我想來點吃到飽"})['result'])
# print(qa_chain({"query": "吃到飽安排上，哪間店推薦下"})['result'])
# print(qa_chain({"query": "天氣冷該吃個火鍋了吧"})['result'])

In [169]:
queries = [
    '請推薦一間牛肉麵',
    '請推薦一間賣串燒的店家',
    '我想吃炸雞',
    '這附近有沒有賣炒飯的',
    '有沒有吃到飽',
    '我想吃拉麵',
    '安排個附近的素食店吧',
    '想吃鬆餅要去哪裡買',
    '推薦個附近的素食店吧',
    '我想吃滷味',
    '推薦學生滷味的哪些餐點',
    '幫我推薦一間便宜的滷味',
    '麥當勞跟肯德基選哪個',
    '有沒有外送串燒',
    '好久沒吃燒臘了',
    '胖老爹跟麥當勞哪個好',
    '今天我想來點吃到飽',
    '吃到飽安排上，哪間店推薦下',
    '天氣冷該吃個火鍋了吧'
]

retrieved_answer = list(map(lambda x: ask_model(x, return_query=True), queries))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
作為一個美食推薦家,你需要使用位於context中的內容去回答問題,且遵循rule中的規則。

rule
1.只要輸出簡潔的一句話就好
2.盡量不要輸出距離有多遠除非問題有提及遠近相關的詞彙
3.不要輸出ASSISTANT: 
endrule

context
name: 台南牛肉鍋
rating: 4.1
keyword: 火鍋, 牛肉
distance: 1.9318254159036776
summary: 溫體牛, 牛肉炒飯, 湯頭鮮甜, 老闆娘熱情, 環境乾淨
endcontext

問題: 請推薦一間牛肉麵
回答: 

> Finished chain.

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
作為一個美食推薦家,你需要使用位於context中的內容去回答問題,且遵循rule中的規則。

rule
1.只要輸出簡潔的一句話就好
2.盡量不要輸出距離有多遠除非問題有提及遠近相關的詞彙
3.不要輸出ASSISTANT: 
endrule

context
name: 佳香味烤肉
rating: 4.1
keyword: 燒烤, 串燒
distance: 1.448491260583128
summary: 烤肉、CP值高、服務好、口味重、外熟內生
endcontext

問題: 請推薦一間賣串燒的店家
回答: 

> Finished chain.

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
作為一個美食推薦家,你需要使用位於context中的內容去回答問題,且遵循rule中的規則。

rule
1.只要輸出簡潔的一句話就好
2.盡量不要輸出

In [133]:
print('\n'.join(retrieved_answer))

問題: 我想吃滷味, 回答: 川醉湘麻辣食堂是一家位於彰化市中正店的美食店,提供多種滷味和其他菜品。這家店的滷味口感好,清洗得乾淨,價格也不貴。此外, 川醉湘麻辣食堂提供多種選擇, 包括滷味和其他菜品。如果你想吃滷味, 我會推薦川醉湘麻辣食堂。
問題: 我想吃滷味, 回答: 劉家滷味是一家平價的小吃店，提供美味的滷味和其他小吃。這裡的氛圍親切，食物也非常好吃。店面乾淨衛生，醬料獨特，是一家值得去品嚐的地方。
問題: 我想吃滷味, 回答: 你可以去食神滷味 - 彰化師大店試試看,那裡的滷味非常好吃!
問題: 我想吃滷味, 回答: 太泰滷味 Super Thai (彰化華山壹號店)是一家提供多樣食材的台式料理和泰式料理的餐廳。他們以平價的方式提供美味的湯頭，其中包括雞肉嫩的菜品。
問題: 我想吃滷味, 回答: 你可以去大滷桶魯味-彰化永安店,那裡的滷味非常好吃。


In [178]:
retrieved_answer = list(map(lambda x: ask_model(x, return_query=True), ['誒你過來一下我要吃關東煮'] * 5))

print('\n'.join(retrieved_answer))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
作為一個美食推薦家,你需要使用位於context中的內容去回答問題,且遵循rule中的規則。

rule
1.只要輸出簡潔的一句話就好
2.盡量不要輸出距離有多遠除非問題有提及遠近相關的詞彙
3.不要輸出ASSISTANT: 
endrule

context
name: 皇后日式關東煮
rating: 4.5
keyword: 日式料理
distance: 1.0930110358665834
summary: 關東煮, 現點現煮, 日式風格, 柴魚湯頭, 食材新鮮
endcontext

問題: 誒你過來一下我要吃關東煮
回答: 


In [7]:
print(qa_chain({"query": "請推薦一間賣串燒的店家"})['result'])

 串串物語是一家平價、好吃的串燒店，他們的醬料非常獨特，不會感到油膩。


In [6]:
print(qa_chain({"query": "我想吃炸雞"})['result'])

 你可以去試試位於彰化華山的胖老爹美式炸雞,那裡有現炸的美式料理和炸雞。


In [7]:
print(qa_chain({"query": "這附近有沒有賣炒飯的"})['result'])

 是的，賞口飯在0.2331710618987155公里外。


In [8]:
print(qa_chain({"query": "有沒有吃到飽"})['result'])

 泰和自助餐是一家平價的餐廳, 提供多樣選擇, 包括蜜汁雞腿。然而, 衛生堪憂, 紅茶喝到飽。


In [9]:
print(qa_chain({"query": "我想吃拉麵"})['result'])

 你可以去台灣四拉麵嚐嚐看,那裡有手工拉麵、麻醬麵、小菜和牛肉麵。


In [10]:
print(qa_chain({"query": "安排個附近的素食店吧"})['result'])

 無想美食館是一家位於1.95公里距離的餐廳,提供現點現做的素食菜品。


In [11]:
print(qa_chain({"query": "想吃鬆餅要去哪裡買"})['result'])

 你可以在順發餅舖購買到美味的古早味芋頭餅。


In [12]:
print(qa_chain({"query": "推薦個附近的素食店吧"})['result'])

 無想美食館是一家位於1.95公里遠的餐廳，提供現點現做的素食菜品。這裡的環境非常舒適，適合聚餐。它的CP值也很高。


In [13]:
print(qa_chain({"query": "我想吃滷味"})['result'])

 你可以去鍋沙滷味，那裡的滷味很好吃。


In [14]:
print(qa_chain({"query": "推薦良叔叔滷味的哪些餐點"})['result'])

 食神滷味 - 彰化師大店是一家以滷味為主打的小吃店,提供多種口味的滷味可選,包括麻辣、清香等。其中最受歡迎的餐點是滷肉和滷鴨蛋,以及烤串和煮豆花也非常好吃。店家的服務態度親切，價格也比較平價。


In [15]:
print(qa_chain({"query": "幫我推薦一間便宜的滷味"})['result'])

 劉家滷味是一家平價、親切的小吃店，他們的滷味非常好吃，而且醬料也很獨特。


In [16]:
print(qa_chain({"query": "麥當勞跟肯德基選哪個"})['result'])

KeyboardInterrupt: 

In [ ]:
print(qa_chain({"query": "有沒有外送串燒"})['result'])

 是的，串串物語提供外送服務。


In [ ]:
print(qa_chain({"query": "好久沒吃燒臘了"})['result'])

 合庫來烤吧是一家提供新鮮食材的居酒屋風格炭火燒烤店, 他們有許多美味的串燒和海鮮可以品嚐。


In [ ]:
print(qa_chain({"query": "胖老爹跟麥當勞哪個好"})['result'])

 鄉村風格, 服務親切, 用餐區在二樓, 餐點品質普通, 薯條炸過頭


In [ ]:
print(qa_chain({"query": "今天我想來點吃到飽"})['result'])

 泰和自助餐是一家平價的餐廳, 提供多樣選擇。他們有蜜汁雞腿, 衛生堪憂, 紅茶喝到飽。


In [ ]:
print(qa_chain({"query": "吃到飽安排上，哪間店推薦下"})['result'])

 泰和自助餐


In [ ]:
print(qa_chain({"query": "天氣冷該吃個火鍋了吧"})['result'])

 三媽臭臭鍋是一家平價的火鍋店, 但他們的湯頭淡而食材少。服務態度也不好, 環境也不太好。
